In [11]:
import numpy as np
import os
import pv_2_3
from pydae.utils import read_data, save_json

In [13]:
from pydae.etools.solar import pv_gen

params = {}
M = 2
N = 3

pfs = [-0.9,1.0,0.9]
hours = [8,9,10,11,12,13,14]
minutes = [0,15,30,45]
day = 180

for pf in pfs:
    for hour in hours:
        for minute in minutes:
            pv = pv_gen()
            # radiation_on_panel_eval(self,Day_of_the_year,Local_time_hours,Local_time_minutes):
            q = pv.radiation_on_panel_eval(day,hour,minute)

            for i_m in range(1,M+1):
                for i_n in range(1,N+1):
                    name = 'LV' + f"{i_m}".zfill(2) + f"{i_n}".zfill(2)
                    irrad_norm = + np.random.normal(loc = q, scale=50.0, size=None)
                    params.update({f'irrad_{name}':irrad_norm})    

            model = pv_2_3.model()
            model.ini(params,'xy_0.json')
            p_pu = model.get_value(f'p_s_{name}')
            s_pu = p_pu/np.abs(pf)
            q_pu = np.sign(pf)*np.sqrt(s_pu**2 - p_pu**2)

            for i_m in range(1,M+1):
                for i_n in range(1,N+1):
                    name = 'LV' + f"{i_m}".zfill(2) + f"{i_n}".zfill(2)
                    irrad_norm = + np.random.normal(loc = q, scale=50.0, size=None)
                    params.update({f'q_s_ppc_{name}':q_pu})    

            model.ini(params,'xy_0.json')

            measurements = {}
            conditions = {}
            grid_data = read_data('pv_2_3.json')

            buse_names = [item['name'] for item in grid_data['buses']]

            # Generators active and reactive power (W and var)
            # Low Voltage side
            for i_m in range(1,M+1):
                for i_n in range(1,N+1):
                    name = 'LV' + f"{i_m}".zfill(2) + f"{i_n}".zfill(2)
                    name_mv = 'MV' + f"{i_m}".zfill(2) + f"{i_n}".zfill(2)
                    S_base = model.get_value(f'S_n_{name}')
                    U_base = grid_data['buses'][buse_names.index(name)]['U_kV']*1000

                    P = model.get_value(f'p_s_{name}')*S_base/grid_data['system']['S_base']
                    Q = model.get_value(f'q_s_{name}')*S_base/grid_data['system']['S_base']
                    U_m = model.get_value(f'V_{name}')

                    S = P + 1j*Q
                    S_m = np.abs(S)
                    I_m = S_m/(U_m)


                    measurements.update({f'P_{name}':P})
                    measurements.update({f'Q_{name}':Q})
                    measurements.update({f'U_{name}':U_m}) # "m" means "module"
                    measurements.update({f'I_{name}_{name_mv}':I_m})    

                    
                    conditions.update({f'irrad_{name}':model.get_value(f'irrad_{name}')})   


            # Feeders
            i_n = 1
            for i_m in range(1,M+1):
                name = 'POIMV_MV' + f"{i_m}".zfill(2) + f"{i_n}".zfill(2) 
                name_node = 'POIMV' 
                P_Feeder_pu,Q_Feeder_pu = model.get_mvalue([f'p_line_{name}',f'q_line_{name}'])
                P =P_Feeder_pu*grid_data['system']['S_base']
                Q =Q_Feeder_pu*grid_data['system']['S_base']
                S_base = model.get_value(f'S_n_{name}')
                U_base = grid_data['buses'][buse_names.index('POIMV')]['U_kV']*1000
                U_m = model.get_value(f'V_POIMV')
                S = P_Feeder_pu + 1j*Q_Feeder_pu
                S_m = np.abs(S)
                I_m = S_m/(U_m)

                #name_alvaro = splitted[0] + '_' + ''.join(name[1:-1].replace('_',''))
                measurements.update({f'P_{name}':P_Feeder_pu})
                measurements.update({f'Q_{name}':Q_Feeder_pu})
                measurements.update({f'U_{name_node}':U_m}) # "m" means "module"
                measurements.update({f'I_{name}':I_m})  


            # Powers and voltages at POI
            P_POI_pu,Q_POI_pu =  model.get_mvalue([f'p_line_POI_GRID',f'q_line_POI_GRID'])
            P = P_POI_pu
            Q = Q_POI_pu
            S_base = model.get_value(f'S_n_{name}')
            U_base = grid_data['buses'][buse_names.index('POI')]['U_kV']*1000
            U_m = model.get_value(f'V_POI') 
            S = -(P_POI_pu + 1j*Q_POI_pu)
            S_m = np.abs(S)
            I_m = S_m/(U_m)


            measurements.update({f'P_POI_POIMV':S.real})
            measurements.update({f'Q_POI_POIMV':S.imag})
            measurements.update({f'U_POI':U_m}) # "m" means "module"
            measurements.update({f'I_POI_POIMV':I_m})  

            day_str = f'{day}'.zfill(3)
            hour_str = f'{hour}'.zfill(2)
            minute_str = f'{minute}'.zfill(2)
            if pf < 0: 
                 pf_sign = 'neg'
            else:
                 pf_sign = 'pos'
                 
            pf_str = f'{int(np.abs(pf)*100)}'.zfill(3) + pf_sign
            
            folder_path = f'../data/pv_2_3_{day_str}_{hour_str}_{minute_str}_pf_{pf_str}'
            os.makedirs(folder_path, exist_ok=True)

            save_json(measurements,file=f'{folder_path}/measurements.json')
            save_json(conditions,file=f'{folder_path}/conditions.json')
